**Projeto 1: Lung Cancer Classification using Computerized Tomography (CT) Data**

**Unidade Curicular:** Laboratório de Inteligência Artificial e Ciência de Dados

**Grupo 7:** 
\
up202206252 Inês Neves 
\
up202205826 Beatriz Fonseca
\
up202208293 Maria Beatriz Moreira

**Introdução**

O projeto realizado tem como objetivo desenvolver um modelo capaz de prever a malignidade de nódulos pulmonares com tamanho superior a 3 mm, a partir de imagens de tomografia computadorizada. 








**Etapas Realizadas**

- Transferência da base de dados LIDC-IDRI, com recurso ao NBIA Data Retriever;
- Extração das *features* radiómicas;
- Transferência da biblioteca pylidc;
- Extração das *features*, utilizando a biblioteca pylidc;
- Criação de um ficheiro .csv (*"data_merged.csv"*) com todas as features extraídas;
- Realização de uma análise explorativa dos dados;
- Realização de *Feature Engineering*;
- Seleção e implementação dos algoritmos a utilizar: *Random Forest*, *SVM*, *XGBoost*;
- Análise dos resultados.

**Bibliotecas Necessárias**

In [1]:
import os
import numpy as np
import pandas as pd
import pylidc as pl
from PIL import Image
import SimpleITK as sitk
import pylidc.utils as ut
from collections import Counter
from radiomics import featureextractor

**Extração de Features**

- **Extração de features utilizando a biblioteca pylidc**


De modo a conseguirmos extrair as *features* dos nódulos pulmonares utilizando a biblioteca *pylidc*, precisamos de fazer um pré-processamento dos dados, de modo a juntar e organizar os dados provenientes de anotações realizadas por diferentes radiologistas. 



Uma vez que a interpretação de cada radiologista em relação a um dado nódulo é subjetiva (o que pode provocar ambiguidade nas anotações), decidimos criar a função *calculate_mode* para calcular a moda para cada uma das seguintes *features*: *subtlety*, *internalStructure*, *calcification*, *sphericity*, *margin*, *lobulation*, *spiculation* e *texture*. 


Utilizamos como medida de decisão a moda, uma vez que ao ser calculada, permite escolher o valor mais recorrente, que tende a ser o mais "aceitável" entre as várias interpretações dos radiologistas. Simultaneamente, utilizar a moda permitiu-nos reduzir a variabilidade do conjunto de dados, pois ao reduzir a variação entre as anotações, obtemos uma medida de consenso, tornando o conjunto de dados mais homogéneo.

Deste modo, a função identifica e retorna o valor mais frequente presente nas anotações para cada característica e, caso haja mais de um valor com a frequência máxima (ou seja, em casos de empate), a função escolhe o maior de entre esses valores, para manter a consistência e uniformidade no conjunto de dados final. 

Referências:
https://insightsimaging.springeropen.com/articles/10.1186/s13244-024-01795-5



In [19]:
#Função para calcular a moda das anotações (em caso de empate retorna o valor mais alto)

def calculate_mode(values):
    count = Counter(values)             
    max_freq = max(count.values())      #Encontrar a maior frequência
    modes = [k for k, v in count.items() if v == max_freq]      #Encontrar todos os valores que têm essa frequência
    return max(modes)                   

Relativamente à malignidade (*malignacy*), decidimos definir uma função específica *calculate_mode_malign* para calcular a sua moda, com o objetivo de melhorar a avaliação do risco associado ao nódulo. 

A malignidade é uma das características mais críticas para o diagnóstico, uma vez que representa a probabilidade de um dado nódulo ser maligno. Os valores desta variável vão de 1 (pouco provável) a 5 (muito provável de ser maligno). Existe também um valor intermediário de 3 (que indica uma avaliação "indeterminada"), utilizado pelos radiologistas caso não consigam determinar com precisão o risco associado ao nódulo.

Devido à ambiguidade associada ao valor 3, optamos por minimizar ao máximo os casos em que a malignidade é indeterminada. Para isso, definimos a função de modo que o valor 3 seja aceite como valor modal apenas quando este é a única moda e  reflete a concordância entre os radiologistas de que a malignidade deve, de facto, ser considerada como indeterminada. 
Decidimos também que nas situações em que a moda inclui o valor 3 junto de outra avaliação (um outro valor presente nas anotações de um outro radiologista), removemos o valor 3, dando prioridade às avaliações que indicam um nível específico de risco.

Tomamos esta decisão com o intuito de definir a malignidade com base nas opiniões dos especialistas que sugerem um nível mais objetivo de malignidade.

Para além disso, em casos de empate que não incluam o valor 3, a função retorna a moda mais alta.

In [ ]:
#Função para calcular a moda da malignidade
def calculate_mode_malign(values):
    count = Counter(values)
    max_freq = max(count.values())                               # Encontra a maior frequência
    modes = [k for k, v in count.items() if v == max_freq]       # Encontra todos os valores que têm essa frequência

    # Se houver apenas uma moda, retorna-a
    if len(modes) == 1:
        return modes[0]
    elif len(modes) > 1:
        # Ignorar o valor 3 se estiver entre as modas
        if 3 in modes:
            modes.remove(3)
        
        # Se ainda houver mais de uma moda após remover o 3
        if len(modes) > 1:
            return max(modes)           # Retorna o maior valor entre as modas
        
    return modes[0]

Seguidamente, organizamos a extração e processamento de dados dos nódulos.

In [ ]:
#DataFrame para os nódulos com problemas de agrupamento
problematic_nodules = pd.DataFrame(columns=['patient_id', 'scan_id', 'nodule_id', 'num_annotations'])

#Criação do DataFrame para armazenar os resultados normais
df = pd.DataFrame(columns=[
    'patient_id',
    'scan_id',          
    'nodule_id',
    'num_annotations', 
    'slice_thickness',
    'pixel_spacing',
    'subtlety',
    'internalStructure',
    'calcification',
    'sphericity',
    'margin',
    'lobulation',
    'spiculation',
    'texture',
    'diameter',
    'surface_area',
    'volume',
    'malignancy',
])

#Lista dos atributos para calcular a moda
attributes = ['subtlety', 'internalStructure', 'calcification', 
              'sphericity', 'margin', 'lobulation', 'spiculation', 
              'texture', 'malignancy']

#Extrai os scans
scans = pl.query(pl.Scan).all()     


for scan in scans:
    clusters = scan.cluster_annotations()           # Agrupar as anotações

    for nodule_id, anns in enumerate(clusters):
        num_annotations = len(anns)
        
        #Se existem mais de 4 anotações para um dado nódulo, este é classificado como problemático
        if num_annotations > 4:
            problematic_nodule_entry = pd.DataFrame([{
                'patient_id': anns[0].scan.patient_id,
                'scan_id': anns[0].scan.id,  
                'nodule_id': nodule_id + 1,  
                'num_annotations': num_annotations
            }])
            
            
            problematic_nodules = pd.concat([problematic_nodules, problematic_nodule_entry], ignore_index=True)
            continue  

        #Criar um dicionário para armazenar os atributos do nódulo
        att = dict((col, "") for col in df.columns)
        
        #Usar a primeira anotação para extrair informações do scan e do paciente
        first_annotation = anns[0]
        att['patient_id'] = first_annotation.scan.patient_id
        att['scan_id'] = scan.id  
        att['nodule_id'] = nodule_id + 1
        att['num_annotations'] = num_annotations
        
        #Extrair informações do scan
        att['slice_thickness'] = scan.slice_thickness
        att['pixel_spacing'] = scan.pixel_spacing
        
        #Atribuir a moda dos atributos de interesse para cada nódulo
        for attr in attributes:
            attr_values = [getattr(ann, attr) for ann in anns if getattr(ann, attr) is not None]
            if attr_values == 'malignancy':
                mode = calculate_mode_malign(attr_values)
            else:
                mode = calculate_mode(attr_values)

            att[attr] = mode
        
        #Atribuir valores de diâmetro, área de superfície e volume da primeira anotação
        att['diameter'] = first_annotation.diameter
        att['surface_area'] = first_annotation.surface_area
        att['volume'] = first_annotation.volume
        
        #Adicionar os dados ao DataFrame
        df_entry = pd.DataFrame([att])
        df = pd.concat([df, df_entry], ignore_index=True)

#Guardar o DataFrame 
df.to_csv('features_pylidc.csv', sep=',', index=False)

#Guardar os nódulos problemáticos 
problematic_nodules.to_csv('problematic_nodules.csv', sep=',', index=False)

Da execução do código anterior resultaram dois ficheiros (.csv):

--> "features_pylidc.csv": contém as features extraídas utilizando a biblioteca pylidc. 

--> "problematic_nodules.csv": neste ficheiro estão identificados os nódulos que decidimos classificar como "problemáticos". Visto que apenas existem 4 radiologistas, optamos por não utilizar  os nódulos que apresentavam mais do que 4 anotações para a extração de features. De modo a podermos identificar e analisar as informações desses nódulos, os mesmos foram guardados no ficheiro previamente definido.

In [ ]:
#Visualização dos ficheiro "features_pylidc.csv"

print("features_pylidc.csv")
features_pylidc = pd.read_csv('features_pylidc.csv', index_col=False)
features_pylidc.head()

features_pylidc.csv


,patient_id,scan_id,nodule_id,num_annotations,slice_thickness,pixel_spacing,subtlety,internalStructure,calcification,sphericity,margin,lobulation,spiculation,texture,diameter,surface_area,volume,malignancy
0,LIDC-IDRI-0078,1,1,4,3.0,0.650,5,1,6,4,4,2,2,5,19.500000,1135.239277,2621.823750,3
1,LIDC-IDRI-0078,1,2,4,3.0,0.650,5,1,6,4,2,4,1,5,20.840585,1124.125177,2439.303750,3
2,LIDC-IDRI-0078,1,3,1,3.0,0.650,4,1,5,5,5,1,1,5,5.076662,66.910605,62.107500,1
3,LIDC-IDRI-0078,1,4,4,3.0,0.650,5,1,6,4,2,4,3,5,23.300483,1650.898027,4332.315000,5
4,LIDC-IDRI-0069,2,1,4,2.0,0.741,3,1,6,5,5,5,5,5,12.683877,505.336348,803.854584,3


In [ ]:
#Visualização do ficheiro "problematic_nodules.csv"
print("problematic_nodules.csv")
problematic_nodules = pd.read_csv('problematic_nodules.csv', index_col=False)
print(problematic_nodules.to_string())

problematic_nodules.csv
        patient_id  scan_id  nodule_id  num_annotations
0   LIDC-IDRI-0055       66          1                6
1   LIDC-IDRI-0092      100          2                5
2   LIDC-IDRI-0137      140          3                5
3   LIDC-IDRI-0204      205          1                6
4   LIDC-IDRI-0252      252          1                5
5   LIDC-IDRI-0332      334          1                5
6   LIDC-IDRI-0340      342          1                8
7   LIDC-IDRI-0366      370          1                7
8   LIDC-IDRI-0404      408          5                7
9   LIDC-IDRI-0608      613          6                5
10  LIDC-IDRI-0942      713          1                7
11  LIDC-IDRI-0865      790          1                5
12  LIDC-IDRI-0863      792          2                5
13  LIDC-IDRI-0815      840          1                5


- **Extração de features utilizando o pyradiomics**

In [16]:
# Verifica se a base de dados foi carregada corretamente
scan = pl.query(pl.Scan).first()
if scan:
    print(f"Base de dados carregada com sucesso. Primeiro paciente: {scan.patient_id}")
else:
    print("Erro ao carregar a base de dados.")

Base de dados carregada com sucesso. Primeiro paciente: LIDC-IDRI-0078


In [17]:
#Inicializa o PyRadiomics feature extractor
extractor = featureextractor.RadiomicsFeatureExtractor()
features_list = []

In [ ]:
#Função para criar uma pasta para cada paciente
def create_patient_folder(base_dir, patient_id):
    patient_folder = os.path.join(base_dir, patient_id)
    if not os.path.exists(patient_folder):
        os.makedirs(patient_folder)
    return patient_folder

A função *process_nodule* é definida com o propósito de extrair, processar e guardar imagens e máscaras dos nódulos, para além de calcular as características radiómicas que descrevem cada nódulo.

--> A função gera uma máscara de consenso para o nódulo (*cmask*), delimitando a região de interesse com uma caixa delimitadora ajustada (*cbbox*) que inclui o nódulo com alguma margem de segurança.

--> Para a extração das *features* radiómicas tomamos a decisão de segmentar a imagem do nódulo, aplicando a máscara ao volume da fatia central.

--> Decidimos adotar esta abordagem, uma vez que, após a análise da literatura disponível nas referências seguidamente apresentadas, a fatia central tende a representar melhor as características essenciais do nódulo, evitando viés ou informações redundantes que possam estar presentes em fatias periféricas.

Segundo **Huang et al. (2019)**, a aplicação desta estratégia na deteção de nódulos pulmonares em tomografias computadorizadas (CT) é vantajosa, na medida em que estas capturam as principais características anatómicas de interesse e, simultaneamente, reduzem a quantidade de dados, simplificando assim o seu processamento.
Neste mesmo artigo é enfatizada a praticidade e eficácia de usar fatias centrais para captar informações relevantes, especialmente quando é necessário limitar a quantidade de dados processados.

Para além disso, em **Zhang et al. (2020)** destaca-se que, em diagnósticos de cancro do pulmão o uso de características extraídas de fatias centrais aumenta a *accuracy* dos modelos, pois essa fatia geralmente coincide com o ponto mais volumoso e distintivo do nódulo, onde as características de textura e forma são mais pronunciadas e, por isso, mais úteis para a classificação. Assim, a fatia central não apenas otimiza o tempo e os recursos computacionais, mas também fornece uma base robusta para extrair informações críticas de imagem, podendo melhorar o potencial diagnóstico e a eficácia das análises subsequentes.

**Referências:**
- Huang, Y. et al. (2019). "Lung Nodule Detection in CT Images Using Deep Learning and Locality Sensitive Hashing." International Journal of Biomedical Imaging.

- Zhang, J., et al. (2020). "3D Deep Learning for Lung Cancer Diagnosis." IEEE Access.

In [20]:
#Função para extrair as features com o PyRadiomics
def process_nodule(nodule, vol, patient_folder, node_counter, patient_id, scan_id, features_list):
    cmask, cbbox, masks = ut.consensus(nodule, clevel=0.5, pad=[(30, 30), (30, 30), (0, 0)])

    #Calcula o índice da fatia do meio
    mid_slice_index = (cbbox[2].start + cbbox[2].stop) // 2

    #Extrai a fatia do volume correspondente ao nódulo
    vol_slice = vol[cbbox[0].start:cbbox[0].stop, cbbox[1].start:cbbox[1].stop, mid_slice_index]

    #Extrai a fatia correspondente da máscara
    mask_slice = cmask[:, :, mid_slice_index - cbbox[2].start]

    if mask_slice.max() == 0:
        return  

    #Aplica a máscara à imagem, cortando o nódulo
    nodule_image = np.where(mask_slice > 0, vol_slice, 0)

    #Escala a imagem do nódulo para o intervalo 0-255
    scaled_nodule_image = (nodule_image - nodule_image.min()) / (nodule_image.max() - nodule_image.min()) * 255.0

    #Guarda a imagem do nódulo cortado
    Image.fromarray(np.uint8(scaled_nodule_image)).save(f"{patient_folder}/{node_counter}_node_{mid_slice_index}_slice_image.jpg")

    #Escala a máscara para o intervalo 0-255
    scaled_mask = (mask_slice / mask_slice.max()) * 255.0

    #Guarda a fatia da máscara
    Image.fromarray(np.uint8(scaled_mask)).save(f"{patient_folder}/{node_counter}_node_{mid_slice_index}_slice_mask.jpg")

    #Converter as fatias em imagens SimpleITK para PyRadiomics
    sitk_image = sitk.GetImageFromArray(nodule_image)
    sitk_mask = sitk.GetImageFromArray(mask_slice.astype(np.uint8))  #A máscara precisa ser uint8 para PyRadiomics

    
    features = extractor.execute(sitk_image, sitk_mask, label=1)
 

    #Adicionar informações extras, como patient_id, nodule_id e scan_id às características
    features['patient_id'] = patient_id
    features['nodule_id'] = node_counter 
    features['scan_id'] = scan_id  

    #Adicionar as características à lista de características
    features_list.append(features)

Outra das funções que definimos foi a função *"normalize_hu"*, utilizada para normalizar o volume das imagens de CT, usando as *Hounsfield Units* (HU).

Normalizar as densidades dos tecidos pulmonares em CTs permite uma análise mais consistente e comparável entre exames, realçando estruturas relevantes e reduzindo o impacto de variações externas. Esta padronização facilita a segmentação, a visualização e a aplicação do algoritmo de processamento de imagens.

Com base na informação que analisamos (e que se encontra disponível nas referências inframencionadas), definimos os valores de MIN_HU e MAX_HU de -1200 HU e 800 HU, respetivamente. Escolhemos este intervalo porque abrange as intensidades de HU típicas para as estruturas e tecidos pulmonares. 

Usamos, por isso o *np.clip* como forma de ajustar o volume das tomografias ao intervalo definido. Posteriormente, a normalização para o intervalo 0-1 é aplicada de modo que, os dados sejam consistentes, independente da variação de HU entre diferentes CTs.

Assim, ao limitarmos e normalizarmos o volume para o intervalo anteriormente mencionado, garantimos que apenas as intensidades dentro da faixa de interesse são processadas, o que permite reduzir a presença de ruído nos dados e realçar apenas as variações de intensidade importantes para a análise de nódulos pulmonares.

Para além disso, tal como é referido no artigo que analisamos, o uso de um intervalo consistente de HU ajuda a padronizar e melhorar a precisão de extração de *features*, garantindo o foco nas características pulmonares relevantes.

A implementação deste passo de pré-processamento dos dados foi, por isso, realizada com o objetivo de melhorar a comparabilidade das imagens e, consequentemente, a extração de *features*.


Referências:
https://mlerma54.github.io/papers/lidc-dicom.pdf

In [ ]:
#Função para normalizar o volume usando as Hounsfield Units (HU)
def normalize_hu(volume):
    
    #Define os valores de HU mínimo e máximo
    MIN_HU = -1200
    MAX_HU = 800  
    
    #Clip (limitar) os valores entre MIN_HU e MAX_HU
    volume = np.clip(volume, MIN_HU, MAX_HU)

    #Normaliza os valores para o intervalo 0-1
    volume = (volume - MIN_HU) / (MAX_HU - MIN_HU)
    return volume

Seguidamente, apresentamos o código do *loop* principal que automatiza a extração das *features* usando o *PyRadiomics*.

Para cada paciente, o código localiza e carrega o volume da tomografia, normaliza os valores de intensidade para facilitar a identificação dos nódulos e organiza os dados em pastas específicas.
Depois, deteta e processa cada nódulo e faz a extração das *features*, utilizando a função *process_nodule* definida anteriormente.


In [ ]:
#Main processing loop
base_dir = r"C:\Users\beatr\Music\Lab_IACD\NOVAS_FEATURES" 

for x in range(1012): 
    try:
        #Consulta para obter o scan do paciente
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == f"LIDC-IDRI-{x+1:04d}").first()

        if scan is not None:
            patient_id = f"LIDC-IDRI-{x+1:04d}"
            patient_folder = create_patient_folder(base_dir, patient_id)

            print(f"Processing patient {patient_id}")
            vol = scan.to_volume()

            #Normaliza o volume para Hounsfield Units (HU)
            vol = normalize_hu(vol)

            nods = scan.cluster_annotations()

            if not nods:
                print(f"No annotations for patient {patient_id}")
                continue

            #Loop por todas as anotações agrupadas (nódulos)
            for nodule_id, anns in enumerate(nods, start=1):  
                try:
                    #Processa cada nódulo e extrai características usando PyRadiomics
                    process_nodule(anns, vol, patient_folder, nodule_id, patient_id, scan.id, features_list)

                except Exception as e:
                    print(f"Error processing nodule {nodule_id} for patient {patient_id}: {e}")

    except Exception as e:
        print(f"Error processing patient LIDC-IDRI-{x+1:04d}: {e}")


features_df = pd.DataFrame(features_list)

#Guarda o DataFrame no formato CSV
features_df.to_csv(os.path.join(base_dir, 'features_pyradiomics.csv'), index=False)

print("Processing complete.")

Da execução do código anterior resultou o seguinte ficheiro (.csv):

--> "features_pyradiomics.csv": contém as features extraídas utilizando o pyradiomics. 

In [5]:
#Visualização do ficheiro "features_pyradiomics.csv"
print("features_pyradiomics.csv")
features_pyradiomics = pd.read_csv('features_pyradiomics.csv', index_col=False)
features_pyradiomics.head()

features_pyradiomics.csv


,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,patient_id,nodule_id,scan_id
0,v3.1.0,1.24.3,2.4.0,1.4.1,3.8.20,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5eb52246a7699383ac7847a5a0478d10042d28f2,2D,"(1.0, 1.0)",...,0.001125,0.0,0.0,1000000.0,0.0,0.0,0.0,LIDC-IDRI-0001,1,12
1,v3.1.0,1.24.3,2.4.0,1.4.1,3.8.20,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},361737ea2d011b9f048963a8659d0d8bb56c142f,2D,"(1.0, 1.0)",...,0.001215,0.0,0.0,1000000.0,0.0,0.0,0.0,LIDC-IDRI-0002,1,13
2,v3.1.0,1.24.3,2.4.0,1.4.1,3.8.20,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},602489a9b2d976a60e9c6df170f05b10a19bfc05,2D,"(1.0, 1.0)",...,0.001590,0.0,0.0,1000000.0,0.0,0.0,0.0,LIDC-IDRI-0003,1,14
3,v3.1.0,1.24.3,2.4.0,1.4.1,3.8.20,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},4a5d028a0ba752432577a82d2fe59e34d48a72e6,2D,"(1.0, 1.0)",...,0.001880,0.0,0.0,1000000.0,0.0,0.0,0.0,LIDC-IDRI-0003,2,14
4,v3.1.0,1.24.3,2.4.0,1.4.1,3.8.20,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5733313731e8dc424d3e35a6a5339a4b8706bf5c,2D,"(1.0, 1.0)",...,0.009346,0.0,0.0,1000000.0,0.0,0.0,0.0,LIDC-IDRI-0003,3,14
